# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.32.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4) ## VER
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [5]:
data_link = 'https://raw.githubusercontent.com/fcgomezr/Udacity-Project-3/main/data%20sets/bank-full.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link, separator = ";")

In [6]:
complete_data = input_data.to_pandas_dataframe()
complete_data 

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [7]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
15350,34,technician,married,secondary,no,351,yes,no,cellular,18,jul,467,1,-1,0,unknown,no
12001,55,retired,married,secondary,no,241,yes,no,unknown,20,jun,14,6,-1,0,unknown,no
10099,41,management,single,tertiary,no,8368,no,no,unknown,11,jun,179,1,-1,0,unknown,no
29232,58,retired,married,secondary,no,1227,no,no,cellular,2,feb,277,1,-1,0,unknown,no
18596,42,blue-collar,married,primary,no,0,yes,no,telephone,31,jul,190,4,-1,0,unknown,no


In [8]:
train_data.to_csv("traindata.csv")

In [9]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './traindata.csv')])

Uploading an estimated of 9 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 9
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 9
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 3 files out of an estimated total of 9
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 4 files out of an estimated total of 9
Uploading ./model_training.py
Uploaded ./model_training.py, 5 files out of an estimated total of 9
Uploading ./README.md
Uploaded ./README.md, 6 files out of an estimated total of 9
Uploading ./score.py
Uploaded ./score.py, 7 files out of an estimated total of 9
Uploading ./traindata.csv
Uploaded ./traindata.csv, 8 files out of an estimated total of 9
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 9 files out of an estimated total of 9
Uploaded 9 files


In [10]:
temp = ftrain_data.drop_columns('Columns1')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="y",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_27c618ae-14b9-4be3-a4e3-24d7fa1e905f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_27c618ae-14b9-4be3-a4e3-24d7fa1e905f,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3965                             |yes                              |33908                                 |
+---------------------------------+----------

{'runId': 'AutoML_27c618ae-14b9-4be3-a4e3-24d7fa1e905f',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-16T22:41:50.894627Z',
 'endTimeUtc': '2021-08-16T23:12:59.809628Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-154705","workspace_name":"quick-starts-ws-154705","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Package:azureml-automl-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-core, training version:1.33.0, current version:1.32.0
Package:azureml-dataprep, training version:2.20.1, current version:2.18.0
Package:azureml-dataprep-native, training version:38.0.0, current version:36.0.0
Package:azureml-dataprep-rslex, training version:1.18.0, current version:1.16.1
Package:azureml-dataset-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-defaults, training version:1.33.0, current version:1.32.0
Package:azureml-interpret, training version:1.33.0, current version:1.32.0
Package:azureml-mlflow, training version:1.33.0, current version:1.32.0
Package:azureml-pipeline-core, training version:1.33.0, current version:1.32.0
Package:azureml-responsibleai, training version:1.33.0, current version:1.32.0
Package:azureml-telemetry, training version:1.33.0, current version:1.32.0
Package:azureml-train-automl-client, training version:1.33.0, current version:1.

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    timeseries_param_dict=None,
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2, 0.06666666666666667, 0.2, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.2, 0.06666666666666667]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


In [15]:
print(best_automl)

Run(Experiment: capstone-project,
Id: AutoML_27c618ae-14b9-4be3-a4e3-24d7fa1e905f_40,
Type: azureml.scriptrun,
Status: Completed)


In [17]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

In [18]:
#joblib.dump(best_fit_model,'./outputs/bestmodel.joblib')
#best_class.register_model(model_name='best_fit_automl_model.pkl',model_path='/outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")
#conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
#conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("azureml-train")

# Adds dependencies to PythonSection of myenv
#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-16 23:26:58+00:00 Creating Container Registry if not exists..
2021-08-16 23:36:58+00:00 Registering the environment.
2021-08-16 23:36:59+00:00 Use the existing image.
2021-08-16 23:36:59+00:00 Generating deployment configuration.
2021-08-16 23:37:00+00:00 Submitting deployment to compute..
2021-08-16 23:37:04+00:00 Checking the status of deployment customerservice..
2021-08-16 23:41:13+00:00 Checking the status of inference endpoint customerservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://3549e85b-7881-40ec-9a1f-8050095c3c1b.southcentralus.azurecontainer.io/score
http://3549e85b-7881-40ec-9a1f-8050095c3c1b.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
# Web Service Logs
print(service.get_logs())

2021-08-16T23:41:02,096160300+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-16T23:41:02,092329500+00:00 - iot-server/run 
2021-08-16T23:41:02,103249600+00:00 - rsyslog/run 
2021-08-16T23:41:02,305620900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_3489174eb648a475780c9959ff366072/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-16T23:41:02,835460900+00:00 - iot-server/finish 1 0
2021-08-16T23:41:02,836673000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (67)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 101
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-08-16 23:41:08,636 | root | INFO | Starting up app insights client
logging socket was

In [ ]:
# Delete the service
service.delete()